In [17]:
# weekly_report_matrix_days.py
# Output: Details | Mon | Tue | Wed | Thu | Fri | Sat | Sun | Total
# Accepts source Thursday as Thu/Thur/Thurs

import os, re, glob
import pandas as pd

INPUT_DIR  = r"C:\Users\user\OneDrive\Desktop\Dominos"
INPUT_FILE = r"Mill Park Sales Report w.e.10-Aug-2025.csv"
OUTPUT_DIR = r"C:\Users\user\OneDrive\Desktop\Dominos\Clean"

ROW_PATTERNS = [
    (r"^sales\s*exclusive\s*gst$",            "Sales Exclusive GST"),
    (r"^no\.?\s*of\s*deliver(y|ies|s)$",      "No of Deliveries"),
    (r"^no\.?\s*of\s*pick\s*ups?$",           "No of PickUps"),
    (r"^single\s*deliver(y|ies)?\s*%$",       "Single Deliveries %"),
    (r"^food\s*%?\s*actual$|^food%actual$",   "Food%Actual"),
    (r"^food\s*%?\s*theo$|^food%theo$",       "Food%Theo"),
    (r"^labou?r\s*%$|^labour%$",              "LABOUR%"),
    (r"^uber\s*eats\s*sales$",                "Uber Eats Sales"),
    (r"^uber\s*eats\s*order\s*count$",        "Uber Eats Order Count"),
    (r"^uber\s*eats\s*commission$",           "Uber Eats Commission"),
]

# Output day headers (what you want in the CSV)
OUTPUT_DAYS = ["Mon","Tue","Wed","Thu","Fri","Sat","Sun"]
# Acceptable source variants for each output day
DAY_ALIASES = {
    "Mon": ["Mon"],
    "Tue": ["Tue"],
    "Wed": ["Wed"],
    "Thu": ["Thu", "Thur", "Thurs"],  # <-- handle all Thursday spellings
    "Fri": ["Fri"],
    "Sat": ["Sat"],
    "Sun": ["Sun"],
}

def _strip(s):
    return str(s).strip() if pd.notna(s) else None

def _to_num(x):
    if pd.isna(x): return None
    s = str(x).strip().replace("$","").replace(",","")
    if s.endswith("%"): s = s[:-1].strip()
    if s in ("","-"): return None
    try: return float(s)
    except: 
        try: return float(int(float(s)))
        except: return None

def _load_csv(path):
    return pd.read_csv(path, header=None, dtype=str)

def find_settings(df):
    """Find the 'Setting' header row/col and confirm day headers to its right."""
    day_tokens = set(sum(DAY_ALIASES.values(), []))  # all accepted labels
    for i in range(df.shape[0]):
        for j in range(df.shape[1]):
            val = _strip(df.iat[i,j])
            if val and val.lower() == "setting":
                right = [_strip(df.iat[i, j+k]) for k in range(1, 10) if j+k < df.shape[1]]
                if any(r in day_tokens for r in right):
                    return i, j
    raise ValueError("Cannot find 'Setting' header with day columns to the right.")

def match_label(raw):
    if not raw: return None
    norm = re.sub(r"\s+"," ", raw.lower().strip())
    for pat, name in ROW_PATTERNS:
        if re.search(pat, norm): 
            return name
    return None

def build_matrix(in_path, out_dir):
    raw = _load_csv(in_path)
    hdr_row, start_col = find_settings(raw)

    # Build header index map from the header row
    headers, col_idx = [], {}
    for k in range(0, raw.shape[1]-start_col):
        v = _strip(raw.iat[hdr_row, start_col+k])
        if v:
            headers.append(v)
            col_idx[v] = start_col + k

    # Map each OUTPUT day to whichever source column exists (Thu/Thur/Thurs)
    src_for_output = {}
    for out_day in OUTPUT_DAYS:
        found_src = next((alias for alias in DAY_ALIASES[out_day] if alias in headers), None)
        if not found_src:
            raise ValueError(f"Missing day column for '{out_day}'. Looked for any of: {DAY_ALIASES[out_day]}")
        src_for_output[out_day] = found_src

    # Read rows under Setting (don’t stop at first blank; some sheets have gaps)
    records = []
    for i in range(hdr_row + 1, raw.shape[0]):
        label_raw = _strip(raw.iat[i, start_col])
        if not label_raw:
            continue
        label = match_label(label_raw)
        if label:
            row = {"Details": label}
            total = 0.0
            for out_day in OUTPUT_DAYS:
                src_day = src_for_output[out_day]
                val = _to_num(raw.iat[i, col_idx[src_day]])
                row[out_day] = val
                if val is not None:
                    total += val
            row["Total"] = round(total, 2)
            records.append(row)

    df = pd.DataFrame(records).drop_duplicates(subset=["Details"])
    df = df[["Details"] + OUTPUT_DAYS + ["Total"]]

    os.makedirs(out_dir, exist_ok=True)
    base = os.path.splitext(os.path.basename(in_path))[0]
    out_path = os.path.join(out_dir, f"{base}__matrix_days.csv")
    df.to_csv(out_path, index=False)
    return out_path, df

if __name__=="__main__":
    in_path = os.path.join(INPUT_DIR, INPUT_FILE)
    out_path, df = build_matrix(in_path, OUTPUT_DIR)
    print(f"Saved: {out_path}")


Saved: C:\Users\user\OneDrive\Desktop\Dominos\Clean\Mill Park Sales Report w.e.10-Aug-2025__matrix_days.csv


In [18]:
import pandas as pd

# File paths for your two weekly cleaned files
week1_path = r"C:\Users\user\OneDrive\Desktop\Dominos\Clean\Mill Park Sales Report w.e.03-Aug-2025__matrix_days.csv"
week2_path = r"C:\Users\user\OneDrive\Desktop\Dominos\Clean\Mill Park Sales Report w.e.10-Aug-2025__matrix_days.csv"
output_path = r"C:\Users\user\OneDrive\Desktop\Dominos\two_weeks_long.csv"

# Read CSVs
week1 = pd.read_csv(week1_path)
week2 = pd.read_csv(week2_path)

# Add week labels (change names to match your file)
week1["Week"] = "03-Aug-2025"
week2["Week"] = "10-Aug-2025"

# Keep the columns you need
cols = ["Details","Mon","Tue","Wed","Thu","Fri","Sat","Sun","Total","Week"]
week1 = week1[cols]
week2 = week2[cols]

# Combine
combined = pd.concat([week1, week2])

# Unpivot into long format (best for Power BI)
long_df = combined.melt(
    id_vars=["Week","Details"],
    value_vars=["Mon","Tue","Wed","Thu","Fri","Sat","Sun"],
    var_name="Day",
    value_name="Value"
)

# Save for Power BI
long_df.to_csv(output_path, index=False)
print("✅ Comparison file saved:", output_path)


✅ Comparison file saved: C:\Users\user\OneDrive\Desktop\Dominos\two_weeks_long.csv


In [19]:
# weekly_merge_unpivot.py

import os
import pandas as pd

# === Input Files (replace with your actual file paths) ===
file1 = r"C:\Users\user\OneDrive\Desktop\Dominos\Clean\Mill Park Sales Report w.e.03-Aug-2025__matrix_days.csv"
file2 = r"C:\Users\user\OneDrive\Desktop\Dominos\Clean\Mill Park Sales Report w.e.10-Aug-2025__matrix_days.csv"

# === Output File ===
out_file = r"C:\Users\user\OneDrive\Desktop\Dominos\Clean\weekly_comparison.csv"

# --- Function to load and reshape a single file ---
def load_weekly_csv(path):
    df = pd.read_csv(path)

    # Extract week ending from filename
    base = os.path.basename(path)
    if "w.e." in base.lower():
        week = base.lower().split("w.e.")[-1].replace("__matrix_days.csv","").strip()
    else:
        week = base

    # Standardize Thursday column
    df = df.rename(columns=lambda c: c.replace("Thur","Thu").replace("Thurs","Thu"))

    # Ensure expected columns exist
    days = ["Mon","Tue","Wed","Thu","Fri","Sat","Sun"]
    keep_cols = ["Details"] + [d for d in days if d in df.columns]
    df = df[keep_cols]

    # Unpivot Mon–Sun
    df = df.melt(id_vars="Details", value_vars=days, var_name="Day", value_name="Value")

    # Add Week column
    df["Week"] = "W/E " + week

    return df

# --- Load both weeks ---
df1 = load_weekly_csv(file1)
df2 = load_weekly_csv(file2)

# --- Combine ---
merged = pd.concat([df1, df2], ignore_index=True)

# --- Drop rows with missing values ---
merged = merged.dropna(subset=["Value"])

# --- Convert to numeric ---
merged["Value"] = pd.to_numeric(merged["Value"], errors="coerce")

# --- Add DaySort for ordering ---
day_order = {"Mon":1,"Tue":2,"Wed":3,"Thu":4,"Fri":5,"Sat":6,"Sun":7}
merged["DaySort"] = merged["Day"].map(day_order)

# --- Save output ---
merged.to_csv(out_file, index=False)

print(f"Saved merged file: {out_file}")
print(merged.head(15))


Saved merged file: C:\Users\user\OneDrive\Desktop\Dominos\Clean\weekly_comparison.csv
                  Details  Day    Value             Week  DaySort
0     Sales Exclusive GST  Mon  2605.96  W/E 03-aug-2025        1
1        No of Deliveries  Mon    38.00  W/E 03-aug-2025        1
2           No of PickUps  Mon    57.00  W/E 03-aug-2025        1
3     Single Deliveries %  Mon    68.00  W/E 03-aug-2025        1
4             Food%Actual  Mon    27.40  W/E 03-aug-2025        1
5               Food%Theo  Mon    29.59  W/E 03-aug-2025        1
6                 LABOUR%  Mon    35.44  W/E 03-aug-2025        1
7         Uber Eats Sales  Mon   641.00  W/E 03-aug-2025        1
8   Uber Eats Order Count  Mon    15.00  W/E 03-aug-2025        1
9    Uber Eats Commission  Mon    98.71  W/E 03-aug-2025        1
10    Sales Exclusive GST  Tue  3336.17  W/E 03-aug-2025        2
11       No of Deliveries  Tue    42.00  W/E 03-aug-2025        2
12          No of PickUps  Tue   114.00  W/E 03-aug-2025

In [21]:
# weekly_merge_unpivot.py

import os
import pandas as pd

# === Input Files (replace with your actual file paths) ===
file1 = r"C:\Users\user\OneDrive\Desktop\Dominos\Clean\Mill Park Sales Report w.e.03-Aug-2025__matrix_days.csv"
file2 = r"C:\Users\user\OneDrive\Desktop\Dominos\Clean\Mill Park Sales Report w.e.10-Aug-2025__matrix_days.csv"

# === Output File ===
out_file = r"C:\Users\user\OneDrive\Desktop\Dominos\Clean\weekly_comparison.csv"

def load_weekly_csv(path):
    df = pd.read_csv(path)

    # Extract week ending from filename
    base = os.path.basename(path)
    if "w.e." in base.lower():
        week = base.lower().split("w.e.")[-1].replace("__matrix_days.csv","").strip()
    else:
        week = base

    # Standardize Thursday column names
    df = df.rename(columns=lambda c: c.replace("Thur","Thu").replace("Thurs","Thu"))

    # Keep only Mon–Sun + Details
    days = ["Mon","Tue","Wed","Thu","Fri","Sat","Sun"]
    keep_cols = ["Details"] + [d for d in days if d in df.columns]
    df = df[keep_cols]

    # Unpivot into long format
    df = df.melt(id_vars="Details", value_vars=days, var_name="Day", value_name="Value")

    # Add Week column
    df["Week"] = "W/E " + week

    return df

# Load both weeks
df1 = load_weekly_csv(file1)
df2 = load_weekly_csv(file2)

# Merge
merged = pd.concat([df1, df2], ignore_index=True)

# Drop blanks
merged = merged.dropna(subset=["Value"])
merged["Value"] = pd.to_numeric(merged["Value"], errors="coerce")

# Save
merged.to_csv(out_file, index=False)

print(f"✅ Saved merged file: {out_file}")
print(merged.head(12))


✅ Saved merged file: C:\Users\user\OneDrive\Desktop\Dominos\Clean\weekly_comparison.csv
                  Details  Day    Value             Week
0     Sales Exclusive GST  Mon  2605.96  W/E 03-aug-2025
1        No of Deliveries  Mon    38.00  W/E 03-aug-2025
2           No of PickUps  Mon    57.00  W/E 03-aug-2025
3     Single Deliveries %  Mon    68.00  W/E 03-aug-2025
4             Food%Actual  Mon    27.40  W/E 03-aug-2025
5               Food%Theo  Mon    29.59  W/E 03-aug-2025
6                 LABOUR%  Mon    35.44  W/E 03-aug-2025
7         Uber Eats Sales  Mon   641.00  W/E 03-aug-2025
8   Uber Eats Order Count  Mon    15.00  W/E 03-aug-2025
9    Uber Eats Commission  Mon    98.71  W/E 03-aug-2025
10    Sales Exclusive GST  Tue  3336.17  W/E 03-aug-2025
11       No of Deliveries  Tue    42.00  W/E 03-aug-2025
